In [3]:
import zipfile
import geopandas as gpd

zip_file_path = 'tl_2020_24_tabblock20.zip'

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files to the current working directory
    zip_ref.extractall()

shapefile_path = 'tl_2020_24_tabblock20.shp'
block_gdf = gpd.read_file(shapefile_path)

In [4]:
import requests

url = "https://transit.land/api/v2/rest/feed_versions/8523f088e05dcd273b369dd4d65eb771c73c0a22/download?apikey=T2Jljq0EQqqe1JMfijkn3RdtVxoYpwuq"
response = requests.get(url)

if response.status_code == 200:
    with open("output.zip", "wb") as f:
        f.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download the file.")

File downloaded successfully.


In [6]:
import zipfile
import os

zip_file_path = 'output.zip'
extract_to_folder = 'transitland'
os.makedirs(extract_to_folder, exist_ok=True)

# Open the ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # Extract all files to the current working directory
    zip_ref.extractall(extract_to_folder)

In [8]:
import pandas as pd

In [7]:
#CODE FOR BLOCKS
compressed_csv_file_home = 'md_rac_S000_JT00_2020.csv.gz'
compressed_csv_file_work = 'md_wac_S000_JT00_2020.csv.gz'
gdf_filtered = block_gdf[block_gdf['COUNTYFP20'] == '510']

df_origins_blocks = pd.read_csv(compressed_csv_file_home)
df_destinations_blocks = pd.read_csv(compressed_csv_file_work)

df_origins_blocks['h_geocode'] = df_origins_blocks['h_geocode'].astype(str)
df_destinations_blocks['w_geocode'] = df_destinations_blocks['w_geocode'].astype(str)

df_origins_blocks = df_origins_blocks[(df_origins_blocks['h_geocode'].str[:5] == '24510') & (df_origins_blocks['C000'] != 0)]
df_destinations_blocks = df_destinations_blocks[(df_destinations_blocks['w_geocode'].str[:5] == '24510') & (df_destinations_blocks['C000'] != 0)]

df_origins_blocks['h_geocode'] = df_origins_blocks['h_geocode'].astype(int)
df_destinations_blocks['w_geocode'] = df_destinations_blocks['w_geocode'].astype(int)

gdf_filtered['GEOID20'] = gdf_filtered['GEOID20'].astype(int)
origins_merged_blocks = gdf_filtered.merge(df_origins_blocks, left_on = 'GEOID20', right_on = 'h_geocode')
destinations_merged_blocks = gdf_filtered.merge(df_destinations_blocks, left_on = 'GEOID20', right_on = 'w_geocode')

#origins_merged = origins_merged[['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry', 'h_geocode', 'C000']]
#destinations_merged = destinations_merged[['STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'GEOID', 'NAMELSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON', 'geometry', 'w_geocode', 'C000']]

origins_merged_blocks['id'] = origins_merged_blocks.index
destinations_merged_blocks['id'] = destinations_merged_blocks.index

/Users/tommyli/miniconda3/envs/myenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [6]:
import pandas as pd
from shapely.geometry import Point

# Convert latitude and longitude columns to float data types
origins_merged_blocks['INTPTLAT20'] = origins_merged_blocks['INTPTLAT20'].astype(float)
origins_merged_blocks['INTPTLON20'] = origins_merged_blocks['INTPTLON20'].astype(float)
origins_merged_blocks['geometry'] = origins_merged_blocks.apply(lambda row: Point(row['INTPTLON20'], row['INTPTLAT20']), axis=1)

destinations_merged_blocks['INTPTLAT20'] = destinations_merged_blocks['INTPTLAT20'].astype(float)
destinations_merged_blocks['INTPTLON20'] = destinations_merged_blocks['INTPTLON20'].astype(float)
destinations_merged_blocks['geometry'] = destinations_merged_blocks.apply(lambda row: Point(row['INTPTLON20'], row['INTPTLAT20']), axis=1)

In [9]:
import pandas as pd
accurate_times = pd.read_csv('bus_accurate_data.csv')
accurate_times['scheduled_visit_time'] = pd.to_datetime(accurate_times['scheduled_visit_time'])
accurate_times['date'] = accurate_times['scheduled_visit_time'].dt.date
unique_dates = accurate_times['date'].unique()
filtered_dfs = {}
for date in unique_dates:
    filtered_dfs[date] = accurate_times[accurate_times['date'] == date]


/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/1744365064.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  accurate_times = pd.read_csv('bus_accurate_data.csv')


In [10]:
import pandas as pd
from datetime import datetime

gtfs_stop_times = pd.read_csv('transitland/stop_times.txt')

for date, filtered_df in filtered_dfs.items():
    merged_df = gtfs_stop_times.merge(filtered_df[['trip_id', 'stop_id', 'observed_visit_time', 'scheduled_visit_time', 'trip_distance_traveled']], on=['trip_id', 'stop_id'], how="left")

    scheduled_mask = merged_df['scheduled_visit_time'].notna()
    merged_df.loc[scheduled_mask, ['arrival_time', 'departure_time']] = merged_df.loc[scheduled_mask, 'scheduled_visit_time']
    merged_df.loc[~scheduled_mask, ['arrival_time', 'departure_time']] = ''

    merged_df['arrival_time'] = pd.to_datetime(merged_df['arrival_time'])
    merged_df['departure_time'] = pd.to_datetime(merged_df['departure_time'])

    time_mask = (merged_df['arrival_time'].dt.hour >= 7) & (merged_df['arrival_time'].dt.hour < 10)
    filtered_times = merged_df[time_mask]

    filtered_times['arrival_time'] = filtered_times['arrival_time'].dt.strftime('%H:%M:%S')
    filtered_times['departure_time'] = filtered_times['departure_time'].dt.strftime('%H:%M:%S')

    #filtered_times.drop(columns=['scheduled_visit_time'], inplace=True)
    filtered_times['scheduled_visit_time'] = pd.to_datetime(filtered_times['scheduled_visit_time'])
    filtered_times['scheduled_visit_time'] = filtered_times['scheduled_visit_time'].dt.strftime('%H:%M:%S')

    observed_mask = filtered_times['observed_visit_time'].notna()
    filtered_times.loc[observed_mask, ['arrival_time', 'departure_time']] = filtered_times.loc[observed_mask, 'observed_visit_time']
    filtered_times.loc[~observed_mask, ['arrival_time', 'departure_time']] = ''

    filtered_times['arrival_time'] = pd.to_datetime(filtered_times['arrival_time'])
    filtered_times['departure_time'] = pd.to_datetime(filtered_times['departure_time'])

    filtered_times['arrival_time'] = filtered_times['arrival_time'].dt.strftime('%H:%M:%S')
    filtered_times['departure_time'] = filtered_times['departure_time'].dt.strftime('%H:%M:%S')

    filtered_times.drop(columns=['observed_visit_time'], inplace=True)

    filtered_times.to_csv(f'stop_times/stop_times_{date}.txt', index=False)


/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/109033976.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_times['arrival_time'] = filtered_times['arrival_time'].dt.strftime('%H:%M:%S')
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/109033976.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_times['departure_time'] = filtered_times['departure_time'].dt.strftime('%H:%M:%S')
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/109033976.p

In [8]:
# for date, filtered_df in filtered_dfs.items():
#     gtfs_stop_times = pd.read_csv('transitland/stop_times.txt')
#     gtfs_stop_times = gtfs_stop_times.merge(filtered_df[['trip_id', 'stop_id', 'observed_visit_time', 'scheduled_visit_time']], on=['trip_id', 'stop_id'], how = "left")

#     observed_mask = gtfs_stop_times['scheduled_visit_time'].notna()
#     gtfs_stop_times.loc[observed_mask, ['arrival_time', 'departure_time']] = gtfs_stop_times.loc[observed_mask, 'scheduled_visit_time']
#     gtfs_stop_times.loc[~observed_mask, ['arrival_time', 'departure_time']] = ''

#     gtfs_stop_times['arrival_time'] = pd.to_datetime(gtfs_stop_times['arrival_time'])
#     gtfs_stop_times['departure_time'] = pd.to_datetime(gtfs_stop_times['departure_time'])

#     gtfs_stop_times['arrival_time'] = gtfs_stop_times['arrival_time'].dt.strftime('%H:%M:%S')
#     gtfs_stop_times['departure_time'] = gtfs_stop_times['departure_time'].dt.strftime('%H:%M:%S')

#     gtfs_stop_times.drop(columns=['observed_visit_time'], inplace=True)

#     gtfs_stop_times.to_csv(f'transitland/stop_times_{date}.txt', index=False)

In [11]:
import os

current_folder = os.getcwd()  # Get the current working directory (where your IPython Notebook is located)
stop_times_folder = os.path.join(current_folder, 'stop_times')

folder_path = stop_times_folder
file_paths = []

# Loop through the files in the folder
for file in os.listdir(folder_path):
    if file.endswith(".txt"):
        file_paths.append(os.path.join(folder_path, file))



In [12]:
def fill_missing_times(trip_df):
    trip_df["arrival_time"] = pd.to_datetime(trip_df["arrival_time"])
    trip_df["departure_time"] = pd.to_datetime(trip_df["departure_time"])
    
    if pd.isnull(trip_df.iloc[0]["arrival_time"]):
        trip_df.loc[trip_df.index[0], "arrival_time"] = trip_df.loc[trip_df.index[0], "scheduled_visit_time"]
        trip_df.loc[trip_df.index[0], "departure_time"] = trip_df.loc[trip_df.index[0], "scheduled_visit_time"]

    if pd.isnull(trip_df.iloc[-1]["arrival_time"]):
        trip_df.loc[trip_df.index[-1], "arrival_time"] = trip_df.loc[trip_df.index[-1], "scheduled_visit_time"]
        trip_df.loc[trip_df.index[-1], "departure_time"] = trip_df.loc[trip_df.index[-1], "scheduled_visit_time"]

    known_times = trip_df.dropna(subset=["arrival_time"])
    known_times["arrival_time"] = pd.to_datetime(known_times["arrival_time"])
    
    # Iterate through known time points to calculate and apply estimated times
    for i in range(1, len(known_times)):
        if (known_times.shape[0] < 3): #threshold
            break
        prev_row = known_times.iloc[i - 1]
        next_row = known_times.iloc[i]
        
        # Calculate time difference and speed for this segment
        time_difference = next_row["arrival_time"] - prev_row["arrival_time"]
        distance_difference = next_row["trip_distance_traveled"] - prev_row["trip_distance_traveled"]
        speed = distance_difference / time_difference.total_seconds()
        
        # Find rows with missing arrival times within the current segment
        mask = (trip_df["arrival_time"].isna()) & (trip_df["trip_distance_traveled"] >= prev_row["trip_distance_traveled"]) & (trip_df["trip_distance_traveled"] <= next_row["trip_distance_traveled"])
        
        # Calculate and update estimated arrival and departure times for missing rows within the segment
        trip_df.loc[mask, "arrival_time"] = prev_row["arrival_time"] + pd.to_timedelta((trip_df.loc[mask, "trip_distance_traveled"] - prev_row["trip_distance_traveled"]) / speed, unit="s")
        trip_df.loc[mask, "departure_time"] = trip_df.loc[mask, "arrival_time"]
    trip_df.drop(columns=['scheduled_visit_time'], inplace=True)
    trip_df.drop(columns=['trip_distance_traveled'], inplace=True)
    trip_df['arrival_time'] = pd.to_datetime(trip_df['arrival_time'])
    trip_df['departure_time'] = pd.to_datetime(trip_df['departure_time'])
    trip_df['arrival_time'] = trip_df['arrival_time'].dt.strftime('%H:%M:%S')
    trip_df['departure_time'] = trip_df['departure_time'].dt.strftime('%H:%M:%S')
    return trip_df

for file_path in file_paths:
    result_df = pd.read_csv(file_path)  
    result_df = result_df.groupby("trip_id").apply(fill_missing_times)
    result_df.to_csv(file_path, index=False)  

/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/3911844654.py:44: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  result_df = result_df.groupby("trip_id").apply(fill_missing_times)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/3911844654.py:44: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .gro

In [13]:
import random
import os

def load_data(file_path):
    df = pd.read_csv(file_path)
    return df

def replace_data(trip_id):
    file_rand = random.sample(file_paths, len(file_paths))
    for filePath in file_rand:
        df = load_data(filePath)
        trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
        if trip_id not in trip_ids_to_replace:
            return df.loc[df.trip_id == trip_id] 

updated_folder = "stop_times_updated"
os.makedirs(updated_folder, exist_ok=True)  

for file in file_paths:
    df = load_data(file)
    trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
    for trip_id in trip_ids_to_replace:
        new_data = replace_data(trip_id)
        df = df.loc[df.trip_id != trip_id]
        df = df.append(new_data)
    trip_ids_to_replace = df.loc[df.arrival_time.isna()].trip_id.unique()
    #df.to_csv(f'updated_{file}', index=False)
    updated_file_path = os.path.join(updated_folder, f'updated_{os.path.basename(file)}')
    df.to_csv(updated_file_path, index=False)
    #df.to_csv(os.path.join(folder_path, f'{file}'), index=False)

/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/22217497.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/22217497.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/22217497.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n8cl00000gn/T/ipykernel_40602/22217497.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_data)
/var/folders/3k/_r2_9shs7m9fg1ft4r8n

In [14]:
import os
import shutil
import zipfile

stop_times_folder = 'stop_times_updated'
transitland_folder = 'transitland'
zipfiles_folder = 'zipfiles'

for filename in os.listdir(stop_times_folder):
    if filename.endswith('.txt'):
        with open(os.path.join(stop_times_folder, filename), 'r') as file:
            content = file.read()

        transitland_filepath = os.path.join(transitland_folder, 'stop_times.txt')
        with open(transitland_filepath, 'w') as file:
            file.write(content)

        zip_filename = f'{os.path.splitext(filename)[0]}.zip'
        zip_path = os.path.join(zipfiles_folder, zip_filename)
        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(transitland_folder):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, transitland_folder)
                    zipf.write(file_path, arcname)

        print(f'Processed {filename}')


Processed updated_stop_times_2023-06-20.txt
Processed updated_stop_times_2023-04-26.txt
Processed updated_stop_times_2023-01-17.txt
Processed updated_stop_times_2023-03-06.txt
Processed updated_stop_times_2023-03-17.txt
Processed updated_stop_times_2023-03-03.txt
Processed updated_stop_times_2023-04-20.txt
Processed updated_stop_times_2023-03-28.txt
Processed updated_stop_times_2023-02-26.txt
Processed updated_stop_times_2023-05-10.txt
Processed updated_stop_times_2023-02-09.txt
Processed updated_stop_times_2023-02-08.txt
Processed updated_stop_times_2023-02-06.txt
Processed updated_stop_times_2023-05-30.txt
Processed updated_stop_times_2023-02-14.txt
Processed updated_stop_times_2023-02-28.txt
Processed updated_stop_times_2023-02-15.txt
Processed updated_stop_times_2023-05-22.txt
Processed updated_stop_times_2023-01-20.txt
Processed updated_stop_times_2023-06-01.txt
Processed updated_stop_times_2023-04-07.txt
Processed updated_stop_times_2023-06-02.txt
Processed updated_stop_times_202

In [16]:
%%time
# Allow 6 GB memory
import sys
sys.argv.append(["--max-memory", "6G"])

import datetime
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransitMode, LegMode

# # Filepath to OSM data
osm_fp = "maryland-latest.osm.pbf"
travel_time_matrix = []

zipfiles_folder = 'zipfiles_test'
for zip_filename in os.listdir(zipfiles_folder):
    print(zip_filename)
    if zip_filename.endswith('.zip'):
        # zip_path = os.path.join(zipfiles_folder, zip_filename)
        transport_network = TransportNetwork(
            osm_fp,
            [
                zip_filename,
            ]
        )
        travel_time_matrix_computer = TravelTimeMatrixComputer(
            transport_network,
            origins=origins_merged_blocks,
            destinations=destinations_merged_blocks,
            departure=datetime.datetime(2023,1,19,8,30),
            transport_modes=[TransitMode.BUS, LegMode.WALK],
        )
        travel_time_matrix_blocks = travel_time_matrix_computer.compute_travel_times()
        travel_time_matrix.append(travel_time_matrix_blocks)

updated_stop_times_2023-01-19.zip


com.conveyal.gtfs.GtfsLibException: com.conveyal.gtfs.GtfsLibException: Error loading GTFS.

In [17]:
import pandas as pd
import matplotlib.pyplot as plt

file_paths = ['transitland/stop_times_{}.txt'.format(date) for date in unique_dates]

data_frames = [pd.read_csv(file_path) for file_path in file_paths]
combined_df = pd.concat(data_frames, ignore_index=True)

missing_percentages = []

for trip_id in combined_df['trip_id'].unique():
    group = combined_df[combined_df['trip_id'] == trip_id]
    missing_percentage = (group['arrival_time'].isnull() & group['departure_time'].isnull()).mean() * 100
    missing_percentages.append(missing_percentage)
    if (missing_percentage > 80): 
        print(trip_id)

plt.hist(missing_percentages, bins=10, edgecolor='black')
plt.xlabel('Percentage of Missing Data')
plt.ylabel('Frequency')
plt.title('Histogram of Missing Data Percentage by Trip')
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'transitland/stop_times_2023-01-19.txt'